In [392]:
import os
import string
from collections import defaultdict as dd

import numpy as np

In [489]:
initial_encoder = "абвгдежзийклмнопрстуфхцчшщъыьэюя "

In [490]:
PATH_TO_CORPORA = "./corpora"

### Вспомогательные функции

In [ ]:
def read_data(file_name):
    with open(os.path.join(PATH_TO_CORPORA, file_name), encoding='utf-8') as file:
        texts = []
        for line in file:
            if not line.strip():
                continue
            texts.append(line.strip())
    
    return texts

def encode_mapper(encoding):
    mapper = {}
    
    for idx, symbol in enumerate(encoding):
        mapper[symbol.lower()] = initial_encoder[idx] 
    
    return mapper

def encrypt_text(text, encoding, use_encode=True):
    if use_encode:
        mapper = encode_mapper(encoding)
    else:
        mapper = encoding
        
    new_text = ""
    
    for token in text:
        if token.lower() in mapper:
            new_text += mapper[token.lower()]
        else:
            new_text += token.lower()
           
    return new_text
    

def choose_randomly_test_sentences(texts, encoding, num_sent=10):
    start, end = 0, len(texts)
    samples = []
    encrypted_texts = []
    indexes = []
    for idx in range(num_sent):
        sample = texts[random.randint(start, end)].lower()
        encrypted_text = encrypt_text(sample, encoding)
        print("Реальный текст\n", sample + '\n')
        print("Закодированный текст\n", encrypted_text + '\n')
        samples.append(sample)
        encrypted_texts.append(encrypted_text)
        
        del texts[idx]
        
    return samples, encrypted_texts


def calculate_n_gramm_statitics(texts, n=2):
    statistics = dd(int)
    
    for text in texts:
        for idx in range(len(text) - n):
            n_gramm = text[idx:idx + n].lower()
            
            if not n_gramm.strip():
                continue
            
            if all([True if token in initial_encoder else False for token in n_gramm]):
                statistics[n_gramm] += 1
    
    return statistics

###### Создаём шифр

In [492]:
encrypted_tokens = "".join(random.sample(initial_encoder, k=len(initial_encoder)))
encrypted_tokens

'щшбънзгфчхяэвмьжиспыоарйюкцу лдте'

###### Закодируем тестовые предложения и удалим их из тренировночной выборки 

In [493]:
all_texts = read_data('AnnaKarenina.txt')
decrypted_texts, encrypted_texts = choose_randomly_test_sentences(all_texts, encrypted_tokens, 3)

Реальный текст
 алексей александрович одержал блестящую победу в заседании комиссии семнадцатого августа, но последствия этой победы подрезали его. новая комиссия для исследования во всех отношениях быта инородцев была составлена и отправлена на место с необычайною, возбуждаемою алексеем александровичем быстротой и энергией. через три месяца был представлен отчет. быт инородцев был исследован в политическом, административном, экономическом, этнографическом, материальном и религиозном отношениях. на все вопросы были прекрасно изложены ответы, и ответы, не подлежавшие сомнению, так как они не были произведением всегда подверженной ошибкам человеческой мысли, но все были произведением служебной деятельности. ответы все были результатами официальных данных, донесений губернаторов и архиереев, основанных на донесениях уездных начальников и благочинных, основанных, с своей стороны, на донесениях волостных правлений и приходских священников; и потому все эти ответы были несомненны. все те воп

In [494]:
def get_decrypted_mapper(general_freqs, local_freqs):
    general_freqs = sorted(general_freqs.items(), key=lambda x: x[1], reverse=True)
    local_freqs = sorted(local_freqs.items(), key=lambda x: x[1], reverse=True)
    decrypted_mapper = {}
    
    for (gen_token, gen_freq), (local_token, local_freq) in zip(general_freqs, local_freqs):
        print(f"Утверждается что токен -{gen_token.upper()}- c частотой {round(gen_freq, 3)} будет вместо символа -{local_token.upper()}- с частотой {round(local_freq, 3)}")
        decrypted_mapper[local_token] = gen_token
        
    return decrypted_mapper

In [495]:
general_token_freq = calculate_n_gramm_statitics(all_texts, 1)
local_token_freq = calculate_n_gramm_statitics(encrypted_texts, 1)

In [496]:
print(f"Глобальная частота n_gramm - {general_token_freq}\n")
print(f"Локальная частота n_gramm - {local_token_freq}")

Глобальная частота n_gramm - defaultdict(<class 'int'>, {'а': 117101, 'н': 98139, 'к': 48460, 'р': 56288, 'е': 123648, 'и': 93874, 'о': 162409, 'д': 41632, 'з': 23121, 'с': 75123, 'м': 40526, 'ы': 26214, 'х': 10986, 'т': 84638, 'в': 66559, 'л': 70913, 'ь': 27851, 'г': 25693, 'ч': 23857, 'я': 30434, 'ш': 12068, 'й': 14861, 'ф': 1781, 'п': 34092, 'ж': 16020, 'у': 38129, 'э': 5019, 'ц': 3992, 'ю': 8812, 'б': 24718, 'щ': 4054, 'ъ': 412})

Локальная частота n_gramm - defaultdict(<class 'int'>, {'х': 218, 'э': 136, 'щ': 79, 'с': 190, 'ч': 33, 'ь': 494, 'д': 226, 'ю': 105, 'ц': 141, 'ф': 314, 'м': 154, 'р': 249, 'и': 44, 'п': 26, 'в': 46, 'я': 150, 'к': 48, 'а': 9, 'ы': 49, 'ш': 19, 'т': 72, 'е': 37, 'н': 93, 'ъ': 11, 'ж': 43, 'л': 15, 'у': 72, 'й': 29, 'б': 24, 'з': 3, 'о': 29, 'г': 2})


###### Посмотрим матчинг между символами из глобальной статистики и локальной

1. Отсортируем частоты букв из глобального и тестового корпуса по убыванию.
2. Сопоставим символы из разных корпусов по рейтингу частот.

In [497]:
decrypted_mapper = get_decrypted_mapper(general_token_freq, local_token_freq)

Утверждается что токен -О- c частотой 162409 будет вместо символа -Ь- с частотой 494
Утверждается что токен -Е- c частотой 123648 будет вместо символа -Ф- с частотой 314
Утверждается что токен -А- c частотой 117101 будет вместо символа -Р- с частотой 249
Утверждается что токен -Н- c частотой 98139 будет вместо символа -Д- с частотой 226
Утверждается что токен -И- c частотой 93874 будет вместо символа -Х- с частотой 218
Утверждается что токен -Т- c частотой 84638 будет вместо символа -С- с частотой 190
Утверждается что токен -С- c частотой 75123 будет вместо символа -М- с частотой 154
Утверждается что токен -Л- c частотой 70913 будет вместо символа -Я- с частотой 150
Утверждается что токен -В- c частотой 66559 будет вместо символа -Ц- с частотой 141
Утверждается что токен -Р- c частотой 56288 будет вместо символа -Э- с частотой 136
Утверждается что токен -К- c частотой 48460 будет вместо символа -Ю- с частотой 105
Утверждается что токен -Д- c частотой 41632 будет вместо символа -Н- с ча

In [498]:
def decrypt_text(encrypted_texts, decryptor):
    for idx, text in enumerate(encrypted_texts):
        new_text = ""
        for token in text:
            if token in decryptor:
                new_text += decryptor[token]
            else:
                new_text += token
                
        print(f"Закодированный текст № {idx}\n".upper(), text + '\n')
        print(f"Расшифрованный текст № {idx}\n".upper(), new_text + '\n')

In [499]:
decrypt_text(encrypted_texts, decrypted_mapper)

ЗАКОДИРОВАННЫЙ ТЕКСТ № 0
 хэ щс чьхэ щсхдюцфмриьфю цпхэьвэ сякаышьтфв юыьмьехс юхдррьщфнрссррьс ндхюъхяфжфьхмжысях,ьдфьтфсэ юсямркьляфчьтфв юуьтфюц ехэрь жф.ьдфмхкьщфнрссркьюэкьрссэ юфмхдркьмфьмс йьфядфб дркйьвуяхьрдфцфюъ мьвуэхьсфсяхмэ дхьрьфятцхмэ дхьдхьн сяфьсьд фвуихчдфш,ьмфевыпюх нфшьхэ щс  ньхэ щсхдюцфмри ньвусяцфяфчьрьлд цжр ч.ьи ц еьяцрьн скъхьвуэьтц юсяхмэ дьфяи я.ьвуяьрдфцфюъ мьвуэьрссэ юфмхдьмьтфэряри сщфн,ьхюнрдрсяцхярмдфн,ьлщфдфнри сщфн,ьлядфжцхзри сщфн,ьнхя црхэодфньрьц эржрфедфньфядфб дркй.ьдхьмс ьмфтцфсуьвуэрьтц щцхсдфьреэфп дуьфям яу,ьрьфям яу,ьд ьтфюэ пхмбр ьсфнд дрш,ьяхщьщхщьфдрьд ьвуэрьтцфрем ю др ньмс жюхьтфюм цп ддфчьфбрвщхньи эфм и сщфчьнусэр,ьдфьмс ьвуэрьтцфрем ю др ньсэып вдфчью кя эодфсяр.ьфям яуьмс ьвуэрьц еыэояхяхнрьфзрърхэодуйьюхддуй,ьюфд с дрчьжыв цдхяфцфмьрьхцйр ц  м,ьфсдфмхддуйьдхьюфд с дркйьы еюдуйьдхихэодрщфмьрьвэхжфирддуй,ьфсдфмхддуй,ьсьсмф чьсяфцфду,ьдхьюфд с дркйьмфэфсядуйьтцхмэ дрчьрьтцрйфюсщрйьсмка ддрщфм;ьрьтфяфныьмс ьлярьфям яуьвуэрьд сфнд дду.ь

## Пункт 2 

###### Подсчитаем статистики по биграммам на всё корпусе, кроме тестовой выборки. Би-граммы с пробелами также учитываем в статистике

In [500]:
general_bi_gramm_stats = calculate_n_gramm_statitics(all_texts)
print(general_bi_gramm_stats)

defaultdict(<class 'int'>, {'ан': 7745, 'нн': 4871, 'на': 18419, 'а ': 23433, ' к': 14933, 'ка': 12953, 'ар': 3934, 'ре': 8822, 'ен': 12233, 'ни': 12996, 'ин': 6910, ' о': 22241, 'од': 7044, 'ди': 3426, 'н ': 7107, ' и': 18461, 'из': 2997, 'з ': 1534, ' с': 26842, 'са': 2977, 'ам': 4036, 'мы': 1551, 'ых': 1385, 'х ': 3382, ' з': 6881, 'зн': 3256, 'ме': 4967, 'ит': 6390, 'ты': 2472, ' р': 6410, 'ро': 11580, 'ом': 8096, 'ма': 4153, 'но': 16321, 'ов': 12277, 'в ': 8149, ' л': 5854, 'ль': 5913, 'ьв': 49, 'ва': 9178, ' т': 13083, 'то': 24155, 'ол': 9205, 'лс': 2024, 'ст': 17652, 'ог': 7738, 'го': 14058, ' н': 27003, 'ач': 1313, 'чи': 2162, 'ае': 1724, 'ет': 7476, 'тс': 1432, 'ся': 5025, 'я ': 14478, 'та': 8313, 'ав': 5119, 'вш': 1782, 'ше': 3776, 'ей': 4194, 'й ': 8969, ' а': 5037, 'аф': 429, 'фо': 164, 'ор': 9113, 'ри': 7390, 'зм': 464, 'мо': 5264, 'м ': 8041, ' ф': 680, 'фр': 419, 'ра': 11537, 'аз': 6998, 'зо': 600, 'ой': 5141, 'вс': 5805, 'се': 7356, 'е ': 25160, 'сч': 718, 'ча': 3731, '

###### Рассчитаем би-граммы на зашифрованных тестовых данных 

In [501]:
local_bi_gramm_stats = calculate_n_gramm_statitics(encrypted_texts)
print(local_bi_gramm_stats)

defaultdict(<class 'int'>, {'хэ': 47, 'э ': 50, ' щ': 32, 'щс': 28, 'с ': 37, ' ч': 14, 'чь': 22, 'ьх': 31, 'сх': 19, 'хд': 34, 'дю': 14, 'юц': 20, 'цф': 40, 'фм': 41, 'мр': 28, 'ри': 19, 'иь': 5, 'ьф': 41, 'фю': 15, 'ю ': 17, ' ц': 23, 'цп': 4, 'пх': 7, 'эь': 12, 'ьв': 24, 'вэ': 2, ' с': 26, 'ся': 40, 'як': 1, 'ка': 2, 'аы': 1, 'ыш': 3, 'шь': 8, 'ьт': 53, 'тф': 25, 'фв': 11, 'в ': 6, ' ю': 23, 'юы': 2, 'ыь': 9, 'ьм': 50, 'мь': 21, 'ье': 9, 'ех': 11, 'хс': 8, 'юх': 15, 'др': 47, 'рр': 11, 'рь': 61, 'ьщ': 19, 'щф': 24, 'фн': 28, 'нр': 14, 'рс': 16, 'сс': 11, 'ср': 13, 'ьс': 39, ' н': 23, 'нд': 7, 'дх': 23, 'хю': 5, 'юъ': 4, 'ъх': 3, 'хя': 9, 'яф': 37, 'фж': 14, 'жф': 19, 'фь': 49, 'хм': 13, 'мж': 1, 'жы': 4, 'ыс': 2, 'ях': 16, 'ьд': 43, 'дф': 45, 'фс': 22, 'сэ': 14, 'юс': 8, 'ям': 17, 'рк': 14, 'кь': 29, 'ьл': 15, 'ля': 13, 'фч': 10, 'юу': 2, 'уь': 13, 'ц ': 36, ' е': 8, 'эр': 20, 'ь ': 6, ' ж': 7, 'мх': 17, 'хк': 4, 'ью': 24, 'юэ': 5, 'эк': 2, 'ьр': 45, 'юф': 18, 'мф': 17, 'мс': 15, ' 

###### Реализуем дешифровку на bigramm-aх
1. Проитерируемся по локальным статистикам биграмм.
2. Если все символы в n_gramm-е нам неизвестны, то просто присваиваем всем символам те значения, 
которые лежит в n_gramme-е глобальных статистик с тем же рейтингом частоты.
3. Если расшифровка для закодированного символа нам уже известна, то будет просто пропускать его.
4. Дополнительно добавим проверку, чтобы не было совпадений у символов, которые являются расшифровками текущим.

In [502]:
def sort_n_gramm_stats(stats):
    stats = sorted(stats.items(), key=lambda x: x[1], reverse=True)
    return stats


def decrypt_n_gramm(global_stats, local_stats):
    #отсортируем частотности n_gramm по убыванию
    global_stats = sort_n_gramm_stats(global_stats)
    local_stats = sort_n_gramm_stats(local_stats)
    
    # иницилизируем дешифратор
    known_symbols = {}
    
    for rating, (n_gramm, freq) in enumerate(local_stats):
        # Если все символы в n_gramm-е нам неизвестны, то просто присваиваем всем символам те значения,
        # которые лежит в n_gramme-е глобальных статистик с тем же рейтингом частоты.
        # Если расшифровка для закодированного символа нам уже известна, то будет просто пропускать его.
        # Дополнительно добавим проверку, чтобы не было совпадений у символов,
        # которые являются расшифровками текущим.
        for s_idx, symbol in enumerate(n_gramm):
            if symbol not in known_symbols and global_stats[rating][0][s_idx] not in known_symbols.values():
                known_symbols[symbol] = global_stats[rating][0][s_idx]
    
    return known_symbols

In [503]:
decrypted_mapper = decrypt_n_gramm(general_bi_gramm_stats, local_bi_gramm_stats)

In [504]:
print(decrypted_mapper)

{'р': 'о', 'ь': ' ', 'т': 'н', 'э': 'е', ' ': 'т', 'х': 'а', 'ф': 'с', 'я': 'л', 'с': 'к', 'д': 'п', 'щ': 'и', 'м': 'ь', 'н': 'д', 'ц': 'р', 'у': 'м', 'ч': 'в', 'ж': 'ы', 'к': 'у', 'в': 'й', 'и': 'э', 'о': 'я', 'б': 'ж', 'ш': 'б', 'ю': 'ш', 'й': 'з', 'ы': 'г', 'ъ': 'х', 'а': 'щ', 'г': 'ф'}


###### Попробуем расшифровать тексты

In [506]:
for idx, (actual_text, encrypted_text) in enumerate(zip(decrypted_texts, encrypted_texts)):
    decrypted_text = encrypt_text(encrypted_text, decrypted_mapper, use_encode=False)
    print(f'Текст № {idx}\n')
    print("ОРИГИНАЛЬНЫЙ ВАРИАНТ\n", actual_text + '\n')
    print("РАСШИФРОВКА\n", decrypted_text + "\n")

Текст № 0

ОРИГИНАЛЬНЫЙ ВАРИАНТ
 алексей александрович одержал блестящую победу в заседании комиссии семнадцатого августа, но последствия этой победы подрезали его. новая комиссия для исследования во всех отношениях быта инородцев была составлена и отправлена на место с необычайною, возбуждаемою алексеем александровичем быстротой и энергией. через три месяца был представлен отчет. быт инородцев был исследован в политическом, административном, экономическом, этнографическом, материальном и религиозном отношениях. на все вопросы были прекрасно изложены ответы, и ответы, не подлежавшие сомнению, так как они не были произведением всегда подверженной ошибкам человеческой мысли, но все были произведением служебной деятельности. ответы все были результатами официальных данных, донесений губернаторов и архиереев, основанных на донесениях уездных начальников и благочинных, основанных, с своей стороны, на донесениях волостных правлений и приходских священников; и потому все эти ответы были несом

### Вывод пункта 2

Результат печален

In [526]:
def calculate_metric_score(text, encoding, n_gramm_stats):
    encrypted_text = encrypt_text(text, encoding)
    local_n_gramm_stats = calculate_n_gramm_statitics([encrypted_text])
    
    metric = 0
    for n_gramm, freq in local_n_gramm_stats.items():
        if n_gramm in n_gramm_stats:
            metric += freq * np.log(n_gramm_stats[n_gramm])
    
    return metric

In [527]:
def permute_encoder(encoder):
    idx1 = random.randint(0, len(list(encoder))-1)
    idx2 = random.randint(0, len(list(encoder))-1)
    
    if idx1 == idx2:
        return permute_encoder(encoder)
    encoder = list(encoder)
    encoder[idx1], encoder[idx2] = encoder[idx2], encoder[idx1]
    
    return "".join(encoder)

### Последовательность действий:
1. Случайно инициализируем кодировщик в виде строки из тех символов, которые мы можем потенциально встретить в тексте.
2. Переставляем местами два случайных символа - это потенциально новый кодировщик.
3. Кодируем сообщение с помощью _текущего_ и _потенциально_ нового кодировщика.
4. На двух закодированных выше сообщениях считаем локальные статистики по n_gramm-ам.
5. Будем оценивать качество кодировщиков по формуле $\sum_{i=0}^{n}LS_{i} * log(GS_{i})$, где $LS_{i}$ - локальные статистики по n_gramm-ам(в нашем случае биграммы), $GS_{i}$ - глобальные статистики по n_gramm-ам на всём корпусе текстов. 
**_Данная метрика будет тем больше, чем ближе наши локальные статистики к истинным глобальным из обучаемого корпуса_**

6. Если у потенциально нового кодировка метрика выше, то говорим, что данный кодировщик становится _текущим_. _Потенциальный_ кодировщик также может перейти в состояние _текущего_, если вероятность $\frac{score_{1}} 
{score_{2}}$ выше случайной из равномерного распределения.
7. Применяем _текущий_ кодировщик на текст, чтобы его расшифровать. 

In [537]:
def mcmc(text, initial_encoder, n_gramm_stats, n_iters):
    curr_encoder = initial_encoder
    best_encoder = ""
    best_score = 0
    
    for iter_ in range(n_iters):
        new_encoder = permute_encoder(curr_encoder)
        curr_metric_score = calculate_metric_score(text, curr_encoder, n_gramm_stats)
        new_metric_score = calculate_metric_score(text, new_encoder, n_gramm_stats)
        diff_score = new_metric_score - curr_metric_score
        transition_prob = min(1, np.exp(diff_score))
        
        if diff_score > 0:
            curr_encoder = new_encoder
            curr_metric_score = new_metric_score
        if curr_metric_score > best_score:
            best_encoder = curr_encoder
            best_score = curr_metric_score
        if transition_prob > random.uniform(0,1):
            curr_encoder = new_encoder
        if iter_ % 100000 == 0:
            print(f'Iter - {iter_}', encrypt_text(text, curr_encoder), sep='\n')
            
    return best_encoder

In [474]:
total_encrypted_texts = " .".join(encrypted_texts)

In [538]:
general_bi_gramm_stats = calculate_n_gramm_statitics(all_texts)
predicted_encoder = mcmc(encrypted_texts[0], initial_encoder, general_bi_gramm_stats, 300000)

Iter - 0
хэ щс чьхэ щсхдюцфмриьфю цпхэьвэ сякаышьтфв юыьмьехс юхдррьщфнрссррьс ндхюъхяфжфьхмжысях,ьдфьтфсэ юсямркьляфчьтфв юуьтфюц ехэрь жф.ьдфмхкьщфнрссркьюэкьрссэ юфмхдркьмфьмс оьфядфб дркоьвуяхьрдфцфюъ мьвуэхьсфсяхмэ дхьрьфятцхмэ дхьдхьн сяфьсьд фвуихчдфш,ьмфевыпюх нфшьхэ щс  ньхэ щсхдюцфмри ньвусяцфяфчьрьлд цжр ч.ьи ц еьяцрьн скъхьвуэьтц юсяхмэ дьфяи я.ьвуяьрдфцфюъ мьвуэьрссэ юфмхдьмьтфэряри сщфн,ьхюнрдрсяцхярмдфн,ьлщфдфнри сщфн,ьлядфжцхзри сщфн,ьнхя црхэйдфньрьц эржрфедфньфядфб дрко.ьдхьмс ьмфтцфсуьвуэрьтц щцхсдфьреэфп дуьфям яу,ьрьфям яу,ьд ьтфюэ пхмбр ьсфнд дрш,ьяхщьщхщьфдрьд ьвуэрьтцфрем ю др ньмс жюхьтфюм цп ддфчьфбрвщхньи эфм и сщфчьнусэр,ьдфьмс ьвуэрьтцфрем ю др ньсэып вдфчью кя эйдфсяр.ьфям яуьмс ьвуэрьц еыэйяхяхнрьфзрърхэйдуоьюхддуо,ьюфд с дрчьжыв цдхяфцфмьрьхцор ц  м,ьфсдфмхддуоьдхьюфд с дркоьы еюдуоьдхихэйдрщфмьрьвэхжфирддуо,ьфсдфмхддуо,ьсьсмф чьсяфцфду,ьдхьюфд с дркоьмфэфсядуоьтцхмэ дрчьрьтцрофюсщроьсмка ддрщфм;ьрьтфяфныьмс ьлярьфям яуьвуэрьд сфнд дду.ьмс ья ьмфтцфсуьфь

<ipython-input-537-9d2f5363c402>:11: RuntimeWarning: overflow encountered in exp
  transition_prob = min(1, np.exp(diff_score))


Iter - 100000
алексей александрович одержал блестящую победу в заседании комиссии семнадцатого августа, но последствия этой победы подрезали его. новая комиссия для исследования во всех отношениях быта инородцев была составлена и отправлена на место с необычайною, возбуждаемою алексеем александровичем быстротой и энергией. через три месяца был представлен отчет. быт инородцев был исследован в политическом, административном, экономическом, этнографическом, материальном и религиозном отношениях. на все вопросы были прекрасно изложены ответы, и ответы, не подлежавшие сомнению, так как они не были произведением всегда подверженной ошибкам человеческой мысли, но все были произведением служебной деятельности. ответы все были результатами официальных данных, донесений губернаторов и архиереев, основанных на донесениях уездных начальников и благочинных, основанных, с своей стороны, на донесениях волостных правлений и приходских священников; и потому все эти ответы были несомненны. все те вопро

KeyboardInterrupt: 

In [543]:
print("Оригинальный текст\n\n", decrypted_texts[0])

Оригинальный текст

 алексей александрович одержал блестящую победу в заседании комиссии семнадцатого августа, но последствия этой победы подрезали его. новая комиссия для исследования во всех отношениях быта инородцев была составлена и отправлена на место с необычайною, возбуждаемою алексеем александровичем быстротой и энергией. через три месяца был представлен отчет. быт инородцев был исследован в политическом, административном, экономическом, этнографическом, материальном и религиозном отношениях. на все вопросы были прекрасно изложены ответы, и ответы, не подлежавшие сомнению, так как они не были произведением всегда подверженной ошибкам человеческой мысли, но все были произведением служебной деятельности. ответы все были результатами официальных данных, донесений губернаторов и архиереев, основанных на донесениях уездных начальников и благочинных, основанных, с своей стороны, на донесениях волостных правлений и приходских священников; и потому все эти ответы были несомненны. все т

##### МСМС поспособствовал хорошему переводу. Лайк!

## Пункт 4 

In [523]:
text = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"
print(text)

დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


In [531]:
georgian_encoder = "".join(list(set(text)) + [" ", "d", 'k', 'r', 'f'])
print(georgian_encoder)

იႾჲႩႣჰႽႲႨჵდქႹჳნႭჇႧჾჱჶႠეႼჂშსხ dkrf


In [534]:
predicted_encoder = mcmc(text, georgian_encoder, general_bi_gramm_stats, 300000)

Iter - 0
крйшищчищшышвкиъмптдйаъчеишйшизмсвшиъмптдйаъчеивкгрвинихвмлмирммуфкъшжигмвмпчеийклгмизпмсшвдваиргмпккищрклмищчищркирыкйдйшизпдщшйаъмишизмйнсшвкитдгрштдйаъчеиудййицдизмрйкыъккисквщкпвмкицдыдъшкигнпрдибмвжигмъксъмижиъшсклмиъкимукфдо
Iter - 10000
если вы вимите норкальный или почти норкальный тедст у этого сообщения доторый легдо прочитать сдорее всего вы все смелали правильно и получите кадсикальный балл за послемнее четвертое замание дурса хотя донечно я ничего не обещаф
Iter - 20000
если вы вимите нордальный или почти нордальный текст у этого сообшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обешаъ
Iter - 30000
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещац
Iter - 40000
если вы вимите 